https://dev.azure.com/tankerkoenig/_git/tankerkoenig-data

```
├── LICENSE
├── Makefile           <- Makefile with commands like `make data` or `make train`
├── README.md          <- The top-level README for developers using this project.
├── data
│   ├── external       <- Data from third party sources.
│   ├── interim        <- Intermediate data that has been transformed.
│   ├── processed      <- The final, canonical data sets for modeling.
│   └── raw            <- The original, immutable data dump.
│
├── docs               <- A default Sphinx project; see sphinx-doc.org for details
│
├── models             <- Trained and serialized models, model predictions, or model summaries
│
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering),
│                         the creator's initials, and a short `-` delimited description, e.g.
│                         `1.0-jqp-initial-data-exploration`.
│
├── references         <- Data dictionaries, manuals, and all other explanatory materials.
│
├── reports            <- Generated analysis as HTML, PDF, LaTeX, etc.
│   └── figures        <- Generated graphics and figures to be used in reporting
│
├── requirements.txt   <- The requirements file for reproducing the analysis environment, e.g.
│                         generated with `pip freeze > requirements.txt`
│
├── setup.py           <- Make this project pip installable with `pip install -e`
├── src                <- Source code for use in this project.
│   ├── __init__.py    <- Makes src a Python module
│   │
│   ├── data           <- Scripts to download or generate data
│   │   └── make_dataset.py
│   │
│   ├── features       <- Scripts to turn raw data into features for modeling
│   │   └── build_features.py
│   │
│   ├── models         <- Scripts to train models and then use trained models to make
│   │   │                 predictions
│   │   ├── predict_model.py
│   │   └── train_model.py
│   │
│   └── visualization  <- Scripts to create exploratory and results oriented visualizations
│       └── visualize.py
│
└── tox.ini            <- tox file with settings for running tox; see tox.readthedocs.io
```


In [90]:
import os
import datetime
import requests
from urllib.parse import urlparse
import pandas as pd
import calendar

from google.cloud import storage

In [92]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [97]:
tk_stations_url_tpl = "https://dev.azure.com/tankerkoenig/362e70d1-bafa-4cf7-a346-1f3613304973/_apis/git/repositories/0d6e7286-91e4-402c-af56-fa75be1f223d/items?path=/stations/{year:n}/{month}/{year:n}-{month}-{day}-stations.csv"
tk_prices_url_tpl = "https://dev.azure.com/tankerkoenig/362e70d1-bafa-4cf7-a346-1f3613304973/_apis/git/repositories/0d6e7286-91e4-402c-af56-fa75be1f223d/items?path=/prices/{year:n}/{month}/{year:n}-{month}-{day}-prices.csv"
tk_prices_filename_tpl = "data_external_tk_prices_{year}-{month}-{day}.csv"
tk_stations_filename_tpl = "data_external_tk_stations_current.csv"

yesterday = datetime.date.today() - datetime.timedelta(days = 1)
day_delta = datetime.timedelta(days=1)
days_range = 30 #0
end_date = datetime.date.today() - datetime.timedelta(days = 1)
start_date = end_date - days_range*day_delta

tk_stations_url_current = tk_stations_url_tpl.format(year=yesterday.year,month=yesterday.month,day=yesterday.day) + "&download=true"

bucket_name="2021.gasprices.cprima.net"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "S:/github.com/cprima/cprimanet/app/phy/jupyter-notebooks/service-account-file.json"

In [108]:
def getDaysSinceYesterdayAsList(days_range):
    """Returns datetime objects since yesterday as a list"""
    retval = []
    end_date = datetime.date.today() - datetime.timedelta(days = 1)
    start_date = end_date - days_range*day_delta
    for i in range((end_date - start_date).days):
        retval.append(end_date - i*day_delta)
    return retval

def formatStringPaddedYMD(url_tpl, tmp_date):
    """Formats a string with padded day and month integers from datetime object"""
    return url_tpl.format(year=tmp_date.year,month=str(tmp_date.month).zfill(2),day=str(tmp_date.day).zfill(2))

def upload_blob_file(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    return

def upload_blob_string(bucket_name, source_string, destination_blob_name):
    """Uploads a string as file to the bucket. Depends on `from google.cloud import storage`"""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_string(source_string)
    return

def requestUrlsAndUploadBlobs(filename, source_string):
    upload_blob_string(bucket_name, source_string, filename)
    return

def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name)
    retval = []
    for blob in blobs:
        retval.append(blob.name)
    return retval

def list_gs_files(endswith="", startswith=""):
    retval = []
    for filename in list_blobs(bucket_name):
        if filename.endswith(endswith) and filename.startswith(startswith):
            retval.append(filename)
    retval.sort(reverse=False)
    return retval

def retrievePricefiles():
    """@todo: rename"""
    existing_files = list_csv_files(endswith="csv")
    for d in getDaysSinceYesterdayAsList(days_range):
        url = formatStringPaddedYMD(tk_prices_url_tpl, d) + "&download=true"
        filename = formatStringPaddedYMD(tk_prices_filename_tpl, d)
        if filename in existing_files:
            pass
        else:
            r = requests.get(url)
            requestUrlsAndUploadBlobs(filename, r.content)
    return

def retrieveStationfile():
    """@todo: rename"""
    url = formatStringPaddedYMD(tk_stations_url_tpl, datetime.date.today() - datetime.timedelta(days = 1)) + "&download=true"
    filename = tk_stations_filename_tpl
    r = requests.get(url)
    requestUrlsAndUploadBlobs(filename, r.content)
    return

def retrieve_data():
    retrievePricefiles()
    retrieveStationfile()

In [106]:
def getPricefilesFromStorage(days_range=30):
    retval = []
    needle_list = []
    for d in getDaysSinceYesterdayAsList(days_range):
        needle_list.append(d.strftime("data_external_tk_prices_%Y-%m-%d.csv"))
    csv_files_list = list_gs_files(endswith="csv", startswith="data_external_tk_prices_")
    for file in csv_files_list:
        if file in needle_list:
            retval.append(file)
    return retval

def make_dataset():
    upload_blob_string(bucket_name, "foo", "foo.txt")
    pass


In [107]:
retrieve_data()
make_dataset(location_id="WOB")

In [111]:
path_stationsA = r'S:\github.com\cprima\cprimanet\dat\tankerkoenig-stations-wolfsburg.csv'

upload_blob_file(bucket_name, path_stationsA, "data_filtered_tk_stations_wolfsburg.csv")
    

In [99]:

filenames = getPricefilesFromStorage(days_range)


list_temp_raw = []
for filename in filenames:
    temp = pd.read_csv('gs://'+bucket_name+'/'+filename, encoding='utf-8')
    list_temp_raw.append(temp)

df_prices = pd.concat(list_temp_raw)

df_prices = df_prices.drop(['dieselchange', 'e5change', 'e10change'], axis=1)
df_prices['date'] = pd.to_datetime(df_prices['date'], errors='coerce', utc=True)
#df_prices['weekday'] = df_prices['date'].dt.day_name()
df_prices['weekday'] = pd.Series(pd.Categorical(df_prices['date'].dt.day_name(), categories=list(calendar.day_name)))
df_prices['quarter'] = df_prices['date'].dt.to_period('Q')






In [100]:
df_prices

,date,station_uuid,diesel,e5,e10,weekday,quarter
0,2021-11-14 23:00:08+00:00,1ffb14eb-b964-42f4-b1cb-1ec5970f96d0,1.569,1.759,1.699,Sunday,2021Q4
1,2021-11-14 23:00:08+00:00,40c801d7-f906-4d2e-35e0-aa5ef69dbd11,1.569,1.759,1.699,Sunday,2021Q4
2,2021-11-14 23:00:08+00:00,77b6aa05-4107-438c-b50a-c1e3cdd2d783,1.528,1.698,0.000,Sunday,2021Q4
3,2021-11-14 23:00:08+00:00,aca2adbf-e74c-4acf-8532-74ea81e09e5c,1.558,0.000,0.000,Sunday,2021Q4
4,2021-11-14 23:00:08+00:00,79900c12-01ef-4ba3-e7e1-81d976f22792,1.569,1.759,1.699,Sunday,2021Q4
...,...,...,...,...,...,...,...
336154,2021-12-14 22:59:08+00:00,00060788-0004-4444-8888-acdc00000004,1.494,1.654,1.594,Tuesday,2021Q4
336155,2021-12-14 22:59:08+00:00,36581a18-7c48-4ade-9131-4f26da00f93a,1.509,1.599,1.539,Tuesday,2021Q4
336156,2021-12-14 22:59:08+00:00,148fcf54-595b-405a-8db8-79d57c7bcbee,1.549,1.689,1.629,Tuesday,2021Q4
336157,2021-12-14 22:59:08+00:00,6d5cb5d9-4fea-4f26-bffe-65796784bf6a,1.489,1.639,1.579,Tuesday,2021Q4


In [126]:
df_stations = pd.concat([pd.read_csv('gs://'+bucket_name+'/'+"data_filtered_tk_stations_wolfsburg.csv", encoding='utf-8')], ignore_index = True)
#df_stations = df_stations.drop(['first_active', 'openingtimes_json'], axis=1)

In [128]:
df_merged = pd.merge(df_prices, df_stations, left_on='station_uuid',right_on='id',how='inner',suffixes=('_left','_right'))
df_merged.index=pd.to_datetime(df_merged['date'], utc=True)

In [129]:
df_merged

,date,station_uuid,diesel_left,e5_left,e10_left,weekday,quarter,id,name,brand,...,place,lat,lng,dist,diesel_right,e5_right,e10_right,isOpen,houseNumber,postCode
date,,,,,,,,,,,,,,,,,,,,,
2021-11-15 03:48:09+00:00,2021-11-15 03:48:09+00:00,e1a15081-254a-9107-e040-0b0a3dfe563c,1.589,1.789,1.729,Monday,2021Q4,e1a15081-254a-9107-e040-0b0a3dfe563c,"Wolfsburg, Grauhorststr. 10",HEM,...,Wolfsburg,52.423626,10.753604,5.4,1.579,1.769,1.709,True,10,38440
2021-11-15 04:48:07+00:00,2021-11-15 04:48:07+00:00,e1a15081-254a-9107-e040-0b0a3dfe563c,1.599,1.799,1.739,Monday,2021Q4,e1a15081-254a-9107-e040-0b0a3dfe563c,"Wolfsburg, Grauhorststr. 10",HEM,...,Wolfsburg,52.423626,10.753604,5.4,1.579,1.769,1.709,True,10,38440
2021-11-15 05:53:08+00:00,2021-11-15 05:53:08+00:00,e1a15081-254a-9107-e040-0b0a3dfe563c,1.609,1.809,1.749,Monday,2021Q4,e1a15081-254a-9107-e040-0b0a3dfe563c,"Wolfsburg, Grauhorststr. 10",HEM,...,Wolfsburg,52.423626,10.753604,5.4,1.579,1.769,1.709,True,10,38440
2021-11-15 06:38:07+00:00,2021-11-15 06:38:07+00:00,e1a15081-254a-9107-e040-0b0a3dfe563c,1.569,1.779,1.719,Monday,2021Q4,e1a15081-254a-9107-e040-0b0a3dfe563c,"Wolfsburg, Grauhorststr. 10",HEM,...,Wolfsburg,52.423626,10.753604,5.4,1.579,1.769,1.709,True,10,38440
2021-11-15 07:44:15+00:00,2021-11-15 07:44:15+00:00,e1a15081-254a-9107-e040-0b0a3dfe563c,1.569,1.759,1.699,Monday,2021Q4,e1a15081-254a-9107-e040-0b0a3dfe563c,"Wolfsburg, Grauhorststr. 10",HEM,...,Wolfsburg,52.423626,10.753604,5.4,1.579,1.769,1.709,True,10,38440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-14 17:36:04+00:00,2021-12-14 17:36:04+00:00,014ad61d-37f1-4efc-9e3e-e54c3650ea77,1.479,1.599,1.539,Monday,2021Q4,014ad61d-37f1-4efc-9e3e-e54c3650ea77,Aral Tankstelle,ARAL,...,Wolfsburg,52.427197,10.771580,4.2,1.629,1.819,1.759,True,117,38440
2021-12-14 18:11:09+00:00,2021-12-14 18:11:09+00:00,014ad61d-37f1-4efc-9e3e-e54c3650ea77,1.469,1.599,1.539,Monday,2021Q4,014ad61d-37f1-4efc-9e3e-e54c3650ea77,Aral Tankstelle,ARAL,...,Wolfsburg,52.427197,10.771580,4.2,1.629,1.819,1.759,True,117,38440
2021-12-14 18:14:08+00:00,2021-12-14 18:14:08+00:00,014ad61d-37f1-4efc-9e3e-e54c3650ea77,1.459,1.599,1.539,Monday,2021Q4,014ad61d-37f1-4efc-9e3e-e54c3650ea77,Aral Tankstelle,ARAL,...,Wolfsburg,52.427197,10.771580,4.2,1.629,1.819,1.759,True,117,38440


In [16]:
#https://stackoverflow.com/a/63566603

from google.cloud import storage
import pandas as pd

storage_client = storage.Client()

buckets_list = list(storage_client.list_buckets())
#bucket_name='my_bucket'
bucket = storage_client.bucket(bucket_name)
blobs = bucket.list_blobs()

list_temp_raw = []
for file in blobs:
    filename = file.name
    if filename.endswith("csv") and filename.startswith("data_external_tk_prices_"):
        temp = pd.read_csv('gs://'+bucket_name+'/'+filename, encoding='utf-8')
        #print(filename, temp.head())
        list_temp_raw.append(temp)

df = pd.concat(list_temp_raw)